### evaluating the output forecastings

output of forecastings format:

    carno	startlap	startrank	endrank	diff	sign	pred_endrank	pred_diff	pred_sign	endlap	pred_endlap
      11	12	31	3.0	5.0	2.0	1	1.0	-2.0	-1	49	58
    
refer to:
    19.RankNet/stage_model_regressor.ipynb
    19.RankNet/RankForecasting-stint-paper-1kpitmodel.ipynb

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#from indycar.model.stint_predictor_fastrun import *
#import indycar.model.stint_simulator as stint
#import indycar.model.stint_simulator_shortterm_pitmodel as stint
#import indycar.model.stint_simulator_paper as stint

import indycar.model.quicktest_simulator as stint

# import all functions 
#from indycar.model.global_variables import _hi
import indycar.model.global_variables as gvar
from indycar.model.quicktest_modules import *

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [2]:
def load_dfout(datafile, basedir = '../result/22.PaperFinal/'):
    datafile = basedir + datafile  
    #with open('laptime_rank_timediff_fulltest-oracle-%s.pickle'%year, 'rb') as f:
    with open(datafile, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        dfout = pickle.load(f, encoding='latin1') 
        
        return dfout[0]
    
def load_dfout_all(datafile):
    #with open('laptime_rank_timediff_fulltest-oracle-%s.pickle'%year, 'rb') as f:
    datafile = '../result/22.PaperFinal/' + datafile  
    with open(datafile, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        dfout = pickle.load(f, encoding='latin1') 
        
        return dfout

## 1. evalute stint restuls

In [30]:
events_info = {
    'Phoenix':(256, 1.022),'Indy500':(500,2.5),'Texas':(372,1.5),
    'Iowa':(268,0.894),'Pocono':(500,2.5),'Gateway':(310,1.25)
}

years = ['2013','2014','2015','2016','2017','2018','2019']
events = [f'Indy500-{x}' for x in years]

events.extend(['Phoenix-2018','Texas-2018','Texas-2019','Pocono-2018','Pocono-2019','Iowa-2018','Iowa-2019',
              'Gateway-2018','Gateway-2019'])

events_id={key:idx for idx, key in enumerate(events)}

dataroot = 'test/'

_train_events = [events_id[x] for x in [f'Indy500-{x}' for x in ['2013','2014','2015','2016','2017']]]
_test_event = 'Gateway-2018'

In [21]:
gvar.events = events
gvar.events_id  = events_id
gvar.events_info = events_info


gvar.maxlap = get_event_info(_test_event)[0]
#gvar.LAPTIME_DATASET = LAPTIME_DATASET

In [22]:
### load ml models
_trim = 0
plen = 2
_context_len = 60

_include_final = True
_include_stintlen = True
#_include_stintlen = False
include_str = '1' if _include_final else '0'
stint_str = '1' if _include_stintlen else ''
#outfile=f'stint-dfout-mlmodels-indy500-tr2013_2017-te2018_2019-end{include_str}-normal-t{_trim}.pickle'
#preddf = load_dfout(outfile)
#outfile=f'stint-dfout-mlmodels-indy500-tr2013_2017-te2018_2019-end{include_str}-oracle-t{_trim}.pickle'
#preddf_oracle = load_dfout(outfile)
version = f'IndyCar-d{len(events)}'
outfile=f'{dataroot}/shortterm-dfout-mlmodels-{version}-end{include_str}-rerank-t{plen}-c{_context_len}.pickle'

In [23]:
#outfile=f'stint-dfout-mlmodels-indy500-tr2013_2017-te2018_2019-end{include_str}-normal-t{_trim}-newtry-tuned.pickle'
#preddf = load_dfout(outfile)
#outfile=f'stint-dfout-mlmodels-indy500-tr2013_2017-te2018_2019-end{include_str}-oracle-t{_trim}-newtry-tuned.pickle'
#preddf_oracle = load_dfout(outfile)

In [ ]:
outfile=f'{dataroot}/stint-dfout-mlmodels-{version}-end{include_str}-normal-t{_trim}-tuned.pickle'
preddf = load_dfout(outfile)
outfile=f'{dataroot}/stint-dfout-mlmodels-{version}-end{include_str}-oracle-t{_trim}-tuned.pickle'
preddf_oracle = load_dfout(outfile)

In [ ]:
dfout = preddf['2018']
for clf in dfout:
    df = dfout[clf]
    print('model:', clf)
    stint.get_evalret(df)


In [ ]:
df = preddf['2018']['svr_lin']
df[df['carno']==12]

In [ ]:
dfout = preddf['2019']
for clf in dfout:
    df = dfout[clf]
    print('model:', clf)
    stint.get_evalret(df)

In [ ]:
# load ranknet results
# outfile=f'stint-dfout-ranknet-indy500-tr2013_2017-te2018_2019-end{include_str}-normal-t{_trim}.pickle'
outfile = 'stint-dfout-ranknet-indy500-timediff-noinlap-nopitage-20182019-alldata.pickle'
ranknetdf, ranknet_acc, ranknet_ret  = load_dfout(outfile)

In [ ]:
ranknetdf['2018'].keys()

In [ ]:
len(ranknet_ret['oracle-TIMEDIFF-2018-noinlap-nopitage'])

In [ ]:
ranknet_ret['oracle-TIMEDIFF-2018-noinlap-nopitage'][0]

In [ ]:
dfout = ranknetdf['2018']
for clf in dfout:
    df = dfout[clf]
    print('model:', clf)
    stint.get_evalret(df)

In [ ]:
dfout = ranknetdf['2019']
for clf in dfout:
    df = dfout[clf]
    print('model:', clf)
    stint.get_evalret(df)

In [ ]:
ranknetdf['2018']['pitmodel_mean']

In [ ]:
### stint result, just cannot do rerank, 
### instead use int()
df = do_rerank(ranknetdf['2018']['pitmodel_mean'],short=False)
stint.get_evalret(df)

### sync with the dataset

In [ ]:
def eval_bydf(testdf, bydf, forcematch=True, force2int=False):
    #collect only records in bydf <carno and startlap>
    cars = set(bydf.carno.values)
    startlaps = {}
    for car in cars:
        startlaps[car] = set(bydf[bydf['carno']==car].startlap.values)
        
    retdf = []
    for car in cars:
        for startlap in startlaps[car]:    
            dfrec = testdf[(testdf['carno']==car) & (testdf['startlap']==startlap)]
            
            #check match
            if forcematch:
                a = testdf[(testdf['carno']==car) & (testdf['startlap']==startlap)].to_numpy().astype(int)
                b = bydf[(bydf['carno']==car) & (bydf['startlap']==startlap)].to_numpy().astype(int)

                if len(a)!=0 and len(b)!=0:
                    #compare 
                    #startrank, endrank
                    if not ((a[0][2] == b[0][2]) and (a[0][3] == b[0][3])):
                        #print('mismatch:', a, b)            
                        continue
            
            retdf.append(dfrec)
        
    dfout = pd.concat(retdf)
    
    if force2int:
        dfdata = dfout.to_numpy().astype(int)
        dfout = pd.DataFrame(dfdata, columns =['carno', 'startlap', 'startrank',    
                                         'endrank', 'diff', 'sign',
                                         'pred_endrank', 'pred_diff', 'pred_sign',
                                         'endlap','pred_endlap'
                                        ])
    
    dfout = dfout.sort_values(by=['carno','startlap'])
    
    print('df size:', len(dfout))
    #return acc
    accret = stint.get_evalret(dfout)[0]
    
    return dfout  , accret


def eval_sync(testdf, errlist, force2int=False):
    """
    eval df result by sync with the errlist detected
    remove the records in errlist
    
    """
    #collect only records in bydf <carno and startlap>
    cars = set(testdf.carno.values)
    startlaps = {}
    for car in cars:
        startlaps[car] = set(testdf[testdf['carno']==car].startlap.values)
        
    retdf = []
    for car in cars:
        for startlap in startlaps[car]:    
            dfrec = testdf[(testdf['carno']==car) & (testdf['startlap']==startlap)]
            
            #check match
            this_rec = [car, startlap]
            if this_rec in errlist:
                continue
            
            retdf.append(dfrec)
        
    dfout = pd.concat(retdf)
    
    if force2int:
        dfdata = dfout.to_numpy().astype(int)
        dfout = pd.DataFrame(dfdata, columns =['carno', 'startlap', 'startrank',    
                                         'endrank', 'diff', 'sign',
                                         'pred_endrank', 'pred_diff', 'pred_sign',
                                         'endlap','pred_endlap'
                                        ])
    
    dfout = dfout.sort_values(by=['carno','startlap'])
    
    print('df size:', len(dfout))
    #return acc
    accret = stint.get_evalret(dfout)[0]
    
    return dfout  , accret

def cmp_df(testdf, bydf):
    """
    df can be different, minor difference for the rank when RankNet removes short ts
    """
    #collect only records in bydf <carno and startlap>
    cars = set(bydf.carno.values)
    startlaps = {}
    for car in cars:
        startlaps[car] = set(bydf[bydf['carno']==car].startlap.values)
        
    err_list = []
    retdf = []
    errcnt = 0
    for car in cars:
        for startlap in startlaps[car]:    
            a = testdf[(testdf['carno']==car) & (testdf['startlap']==startlap)].to_numpy().astype(int)
            b = bydf[(bydf['carno']==car) & (bydf['startlap']==startlap)].to_numpy().astype(int)
            
            if len(a)!=0 and len(b)!=0:
                #compare 
                #startrank, endrank
                if not ((a[0][2] == b[0][2]) and (a[0][3] == b[0][3])):
                    print('mismatch:', a, b)
                    errcnt += 1
                    err_list.append([car, startlap])
            else:
                errcnt += 1
                print('mismatch empty:', a, b)
                err_list.append([car, startlap])
                
                
    print('errcnt:', errcnt)
    return errcnt, err_list

#### problem of df mismatch

In [ ]:
errlist = {}

In [ ]:
errcnt, errlist['2019'] = cmp_df(ranknetdf['2019']['pitmodel_mean'], preddf['2019']['lasso'])
errcnt, errlist['2019']

In [ ]:
errcnt, errlist['2018'] = cmp_df(ranknetdf['2018']['pitmodel_mean'], preddf['2018']['lasso'])
errcnt, errlist['2018']

In [ ]:
#
# Model,SignAcc,MAE,50-Risk,90-Risk
# 
cols = ['Year','Model','SignAcc','MAE','50-Risk','90-Risk']
models = {'currank':'CurRank','rf':'RandomForest','svr_lin':'SVM','xgb':'XGBoost'}

In [ ]:
retdata = []
for year in ['2018','2019']:
    for clf in ['currank','rf','svr_lin','xgb']:
        print('year:',year,'clf:',clf)
        dfout, accret = eval_sync(preddf[year][clf],errlist[year])
        fsamples, ftss = df2samples(dfout)
        _, prisk_vals = prisk_direct_bysamples(fsamples, ftss)
        
        retdata.append([year,models[clf],accret[0], accret[1], prisk_vals[1], prisk_vals[2]])
        
    #ml models -oracle
    for clf in ['rf','svr_lin','xgb']:
        print('year:',year,'clf:',clf)
        dfout, accret = eval_sync(preddf_oracle[year][clf],errlist[year])
        fsamples, ftss = df2samples(dfout)
        _, prisk_vals = prisk_direct_bysamples(fsamples, ftss)
        
        retdata.append([year,models[clf]+'-Oracle',accret[0], accret[1], prisk_vals[1], prisk_vals[2]])
        
    dfout, accret = eval_sync(ranknetdf[year]['pitmodel_mean'], errlist[year],force2int=True)
    #fsamples, ftss = df2samples(dfout)
    fsamples, ftss = runs2samples(ranknet_ret[f'pitmodel-TIMEDIFF-{year}-noinlap-nopitage'],errlist[f'{year}'])
    _, prisk_vals = prisk_direct_bysamples(fsamples, ftss)
    retdata.append([year,'RankNet-MLP',accret[0], accret[1], prisk_vals[1], prisk_vals[2]])
    
    dfout, accret = eval_sync(ranknetdf[year]['oracle_mean'], errlist[year],force2int=True)
    #fsamples, ftss = df2samples(dfout)
    fsamples, ftss = runs2samples(ranknet_ret[f'oracle-TIMEDIFF-{year}-noinlap-nopitage'],errlist[f'{year}'])
    _, prisk_vals = prisk_direct_bysamples(fsamples, ftss)
    retdata.append([year,'RankNet-Oracle',accret[0], accret[1], prisk_vals[1], prisk_vals[2]])
        
        
stint_result = pd.DataFrame(data=retdata, columns=cols)
    

In [ ]:
stint_result

In [ ]:
stint_result.to_csv('stint_experiment_result_v4.csv')

#### analysis of car12, pitstop=94

In [ ]:
dfxs = ranknet_ret[f'pitmodel-TIMEDIFF-2018-noinlap-nopitage']
dfx = pd.concat(dfxs)
df12 = dfx[dfx['carno']==12]


In [ ]:
hdata = df12.pred_endlap
plt.hist(hdata, bins=range(min(hdata),max(hdata)+1))

In [ ]:
print('pitstops:', set(df12.startlap.values))
pit94 = df12[df12['startlap']==93]
observed = pit94[['endlap','endrank']].values


In [ ]:
print(observed[0])

In [ ]:
def bins_labels(bins, **kwargs):
    bin_w = (max(bins) - min(bins)) / (len(bins) - 1)
    #plt.xticks(np.arange(min(bins)+bin_w/2, max(bins), bin_w), bins, **kwargs)
    plt.xticks(np.arange(min(bins)+bin_w/2, max(bins), bin_w), bins, **kwargs)
    plt.xlim(bins[0], bins[-1])

In [ ]:
#fig, axs = plt.subplots(1, 2, figsize=(8, 4))
# https://stackoverflow.com/questions/27083051/matplotlib-xticks-not-lining-up-with-histogram
fig, axs = plt.subplots(2, 1, figsize=(8, 7))
hdata = pit94.pred_endlap.astype(int)

bins = np.arange(min(hdata),max(hdata))
axs[0].hist(hdata, bins - 0.5, align='mid')
axs[0].set_xticks(bins)
axs[0].plot(observed[0][0],2,'*',color='r')
axs[0].set_xlabel('Predicted Lap of Next PitStop')
axs[0].set_ylabel('Probability(%)')


hdata = pit94.pred_endrank
axs[1].hist(hdata, bins=np.arange(0,10)-0.5,label='forecast')
axs[1].set_xticks(np.arange(0,10))
axs[1].plot(observed[0][1],10,'*',color='r',label='observed')
axs[1].set_xlabel('Predicted Rank Posisiton of Next PitStop')
axs[1].set_ylabel('Probability(%)')

plt.legend(prop={'size': 10})
plt.tight_layout()
plt.savefig('pit94_stint_forecast.pdf')

In [ ]:
#fig, axs = plt.subplots(1, 2, figsize=(8, 4))
# https://stackoverflow.com/questions/27083051/matplotlib-xticks-not-lining-up-with-histogram
fig, axs = plt.subplots(1,2, figsize=(12, 3))
hdata = pit94.pred_endlap.astype(int)

bins = np.arange(min(hdata),max(hdata))
axs[0].hist(hdata, bins - 0.5, align='mid')
axs[0].set_xticks(bins)
axs[0].plot(observed[0][0],2,'*',color='r')
axs[0].set_xlabel('Predicted Lap of Next PitStop')
axs[0].set_ylabel('Probability(%)')


hdata = pit94.pred_endrank
axs[1].hist(hdata, bins=np.arange(0,10)-0.5,label='forecast')
axs[1].set_xticks(np.arange(0,10))
axs[1].plot(observed[0][1],10,'*',color='r',label='observed')
axs[1].set_xlabel('Predicted Rank Posisiton of Next PitStop')
axs[1].set_ylabel('Probability(%)')

plt.legend(prop={'size': 10})
plt.tight_layout()
plt.savefig('pit94_stint_forecast_h.pdf')

In [ ]:
np.arange(11)

In [ ]:
bins

In [ ]:
hdata = pit94.pred_endrank
plt.hist(hdata, bins=range(0,10))

In [ ]:
pit94.head(2)

## 2. Evaluation shortterm forecasting results

In [24]:
#
# Model,SignAcc,MAE,50-Risk,90-Risk
# 
cols = ['Year','Model','SignAcc','MAE','50-Risk','90-Risk']
models = {'currank':'CurRank','rf':'RandomForest','svr':'SVM','xgb':'XGBoost'}
#plen=2
#usemeanstr='median'
usemeanstr='mean'
stint_result = {}

_context_len = 60

In [31]:
for plen in [2]:
    outfile=f'{dataroot}/shortterm-dfout-mlmodels-{version}-end{include_str}-rerank-t{plen}-c{_context_len}.pickle'
    shortterm_df = load_dfout(outfile, basedir = './')

    retdata = []
    #for year in ['Indy500-2018','Indy500-2019','Phoenix-2018']:
    for year in [_test_event]:
        
        testevent = year
        gvar.maxlap = get_event_info(testevent)[0]
        
        for clf in ['currank','rf','svr','xgb']:
            print('year:',year,'clf:',clf)
            #dfout, accret = eval_sync(preddf[year][clf],errlist[year])
            dfout = shortterm_df[year][clf]
            accret = stint.get_evalret_shortterm(dfout)[0]

            fsamples, ftss = df2samples(dfout)
            _, prisk_vals = prisk_direct_bysamples(fsamples, ftss)

            retdata.append([year,models[clf],accret[0], accret[1], prisk_vals[1], prisk_vals[2]])
            
        if False:
            #oracle
            outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-oracle-l10-alldata.pickle'
            _all = load_dfout_all(outfile)[0]
            ranknetdf, acc, ret, pret = _all[0],_all[1],_all[2],_all[3]
            allsamples, alltss = get_allsamples(year=year, model='oracle')
            #_, pret[mid]= prisk_direct_bysamples(ret[mid][0][1], ret[mid][0][2])
            _, prisk_vals = prisk_direct_bysamples(allsamples, alltss)

            dfout = do_rerank(ranknetdf[year]['oracle_mean'])
            accret = stint.get_evalret_shortterm(dfout)[0]
            #fsamples, ftss = runs2samples_ex(ranknet_ret[f'oracle-RANK-{year}-inlap-nopitage'],[])
            #_, prisk_vals = prisk_direct_bysamples(fsamples, ftss)
            retdata.append([year,'RankNet-Oracle',accret[0], accret[1], prisk_vals[1], prisk_vals[2]])

            #pitmodel
            outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-alldata.pickle'
            _all = load_dfout_all(outfile)[0]
            ranknetdf, acc, ret = _all[0],_all[1],_all[2]

            dfout = do_rerank(ranknetdf[year]['pitmodel_mean'])
            accret = stint.get_evalret_shortterm(dfout)[0]
            #fsamples, ftss = runs2samples_ex(ranknet_ret[f'pitmodel-RANK-{year}-inlap-nopitage'],[])
            #_, prisk_vals = prisk_direct_bysamples(fsamples, ftss)
            allsamples, alltss = get_allsamples(year=year, model='pitmodel')
            #_, pret[mid]= prisk_direct_bysamples(ret[mid][0][1], ret[mid][0][2])
            _, prisk_vals = prisk_direct_bysamples(allsamples, alltss)        
            retdata.append([year,'RankNet-MLP',accret[0], accret[1], prisk_vals[1], prisk_vals[2]])

            #print('oracle:', ranknet_pret[f'oracle-RANK-2018-inlap-nopitage'])
            #print('pitmodel:', ranknet_pret[f'pitmodel-RANK-2018-inlap-nopitage'])

    stint_result[plen] = pd.DataFrame(data=retdata, columns=cols)
    
    print(stint_result[plen])

year: Gateway-2018 clf: currank
model: acc={0.92}, mae={0.33}, rmse={1.17},r2={0.96}, {236}
            naive: acc={0.92}, mae={0.33}, rmse={1.17},r2={0.96}


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1699: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.03616936523185792, 0.03613187431169014, 0.036094383391522364])
year: Gateway-2018 clf: rf
model: acc={0.50}, mae={1.83}, rmse={2.96},r2={0.73}, {236}
            naive: acc={0.92}, mae={0.33}, rmse={1.17},r2={0.96}


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1699: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.1971928673524381, 0.1971553764322703, 0.19711788551210258])
year: Gateway-2018 clf: svr
model: acc={0.92}, mae={0.33}, rmse={1.17},r2={0.96}, {236}
            naive: acc={0.92}, mae={0.33}, rmse={1.17},r2={0.96}


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1699: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.03621622888206763, 0.036178737961899854, 0.036141247041732084])
year: Gateway-2018 clf: xgb
model: acc={0.14}, mae={3.66}, rmse={5.30},r2={0.13}, {236}
            naive: acc={0.92}, mae={0.33}, rmse={1.17},r2={0.96}
dict_values([0.3955666986901609, 0.39552920776999323, 0.3954917168498255])
           Year         Model   SignAcc       MAE   50-Risk   90-Risk
0  Gateway-2018       CurRank  0.915254  0.334490  0.036132  0.036094
1  Gateway-2018  RandomForest  0.495763  1.827332  0.197155  0.197118
2  Gateway-2018           SVM  0.915254  0.334924  0.036179  0.036141
3  Gateway-2018       XGBoost  0.139831  3.662039  0.395529  0.395492


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1699: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


In [32]:
stint_result[2]

,Year,Model,SignAcc,MAE,50-Risk,90-Risk
0,Gateway-2018,CurRank,0.915254,0.334490,0.036132,0.036094
1,Gateway-2018,RandomForest,0.495763,1.827332,0.197155,0.197118
2,Gateway-2018,SVM,0.915254,0.334924,0.036179,0.036141
3,Gateway-2018,XGBoost,0.139831,3.662039,0.395529,0.395492


In [ ]:
dfs = []
for plen in stint_result:
    df = stint_result[plen]
    df['plen'] = plen
    
    dfs.append(df)
    
alldf = pd.concat(dfs)
    

In [ ]:
alldf

In [ ]:
alldf.to_csv(f'evaluation_result_shortterm_plen2-8_all_v4-c{_context_len}.csv')

## 3. Split by pit-covered-laps and normal laps

define pit covered laps := two laps before and after a pit

split shortterm-results only

In [38]:
# get pit laps, pit-covered-laps
# pitdata[year] = [pitlaps, pitcoveredlaps]
covergap = 1
with open(f'{dataroot}/pitcoveredlaps-alldata-g{covergap}.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    pitdata = pickle.load(f, encoding='latin1') 

In [39]:
def clear_samples(full_samples, full_tss, clearidx):
    """
    clear the laps in clearidx
    """
    
    carlist = full_tss.keys()
    
    for carid, carno in enumerate(carlist):
        forecast = full_samples[carno]
        target = full_tss[carno]
        
        forecast[:, clearidx] = np.nan
        target[clearidx] = np.nan
        
        full_samples[carno] = forecast
        full_tss[carno] = target
        
    return full_samples, full_tss

In [40]:
#
# Model,SignAcc,MAE,50-Risk,90-Risk
# 
cols = ['Year','Model','SignAcc','MAE','50-Risk','90-Risk']
models = {'currank':'CurRank','rf':'RandomForest','svr':'SVM','xgb':'XGBoost'}
#plen=2
#usemeanstr='median'
usemeanstr='mean'
stint_split_result = {}

In [41]:
for laptype in ['normal','pit']:
    
    plen = 2
    
    outfile=f'{dataroot}/shortterm-dfout-mlmodels-{version}-end{include_str}-rerank-t{plen}-c{_context_len}.pickle'
    shortterm_df = load_dfout(outfile, basedir = './')

    retdata = []
    #for year in ['2018','2019']:
    for year in [_test_event]:
        
        testevent = year
        gvar.maxlap = get_event_info(testevent)[0]
        
        # select the set
        pitcoveredlaps = pitdata[year][1]
        normallaps = set([x for x in range(1,201)]) - pitcoveredlaps
        
        if laptype == 'normal':
            sellaps = normallaps
            clearlaps = pitcoveredlaps
        else:
            sellaps = pitcoveredlaps
            clearlaps = normallaps
        
        
        # pitcoveredlaps start idx = 1
        startlaps = [x-plen-1 for x in sellaps]
        #sellapidx = np.array([x-1 for x in sellaps])
        clearidx = np.array([x-1 for x in clearlaps])
        
        for clf in ['currank','rf','svr','xgb']:
            print('year:',year,'clf:',clf)
            
            dfout = shortterm_df[year][clf]
            fsamples, ftss = df2samples(dfout)
            
            fsamples, ftss = clear_samples(fsamples, ftss,clearidx)
            _, prisk_vals = prisk_direct_bysamples(fsamples, ftss)
            
            # split dfout by startlap
            # curlap = int(dfrec.startlap.values[0] + prediction_len)
            dfout = dfout[dfout['startlap'].isin(startlaps)]
            accret = stint.get_evalret_shortterm(dfout)[0]

            retdata.append([year,models[clf],accret[0], accret[1], prisk_vals[1], prisk_vals[2]])
            
        if False:
            #oracle
            outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-oracle-l10-alldata.pickle'
            _all = load_dfout_all(outfile)[0]
            ranknetdf, acc, ret, pret = _all[0],_all[1],_all[2],_all[3]

            dfout = do_rerank(ranknetdf[year]['oracle_mean'])

            allsamples, alltss = get_allsamples(year=year, model='oracle')

            allsamples, alltss = clear_samples(allsamples, alltss,clearidx)
            _, prisk_vals = prisk_direct_bysamples(allsamples, alltss)

            dfout = dfout[dfout['startlap'].isin(startlaps)]
            accret = stint.get_evalret_shortterm(dfout)[0]

            retdata.append([year,'RankNet-Oracle',accret[0], accret[1], prisk_vals[1], prisk_vals[2]])

            #pitmodel
            outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-alldata.pickle'
            _all = load_dfout_all(outfile)[0]
            ranknetdf, acc, ret = _all[0],_all[1],_all[2]

            dfout = do_rerank(ranknetdf[year]['pitmodel_mean'])

            allsamples, alltss = get_allsamples(year=year, model='pitmodel')
            allsamples, alltss = clear_samples(allsamples, alltss,clearidx)
            _, prisk_vals = prisk_direct_bysamples(allsamples, alltss)        

            dfout = dfout[dfout['startlap'].isin(startlaps)]
            accret = stint.get_evalret_shortterm(dfout)[0]

            retdata.append([year,'RankNet-MLP',accret[0], accret[1], prisk_vals[1], prisk_vals[2]])

            #print('oracle:', ranknet_pret[f'oracle-RANK-2018-inlap-nopitage'])
            #print('pitmodel:', ranknet_pret[f'pitmodel-RANK-2018-inlap-nopitage'])

    stint_split_result[laptype] = pd.DataFrame(data=retdata, columns=cols)
    
    print(stint_split_result[laptype])

year: Gateway-2018 clf: currank
dict_values([0.004852312550544923, 0.004852312550544922, 0.004852312550544922])
model: acc={1.00}, mae={0.04}, rmse={0.25},r2={1.00}, {121}
            naive: acc={1.00}, mae={0.04}, rmse={0.25},r2={1.00}
year: Gateway-2018 clf: rf


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1699: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.12454268879731968, 0.12454268879731968, 0.1245426887973197])
model: acc={0.42}, mae={0.98}, rmse={1.64},r2={0.92}, {121}
            naive: acc={1.00}, mae={0.04}, rmse={0.25},r2={1.00}
year: Gateway-2018 clf: svr


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1699: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.004852312550544923, 0.004852312550544922, 0.004852312550544922])
model: acc={1.00}, mae={0.04}, rmse={0.25},r2={1.00}, {121}
            naive: acc={1.00}, mae={0.04}, rmse={0.25},r2={1.00}
year: Gateway-2018 clf: xgb


...

/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1699: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.30153754587838505, 0.3014581886717588, 0.30137883146513245])
model: acc={0.54}, mae={2.86}, rmse={4.06},r2={0.48}, {94}
            naive: acc={0.79}, mae={0.78}, rmse={1.83},r2={0.89}
year: Gateway-2018 clf: svr


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1699: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


dict_values([0.07229441523658367, 0.07221505802995734, 0.07213570082333103])
model: acc={0.79}, mae={0.78}, rmse={1.83},r2={0.89}, {94}
            naive: acc={0.79}, mae={0.78}, rmse={1.83},r2={0.89}
year: Gateway-2018 clf: xgb
dict_values([0.5773038389048707, 0.5772244816982443, 0.5771451244916179])
model: acc={0.16}, mae={5.66}, rmse={7.23},r2={-0.64}, {94}
            naive: acc={0.79}, mae={0.78}, rmse={1.83},r2={0.89}
           Year         Model   SignAcc       MAE   50-Risk   90-Risk
0  Gateway-2018       CurRank  0.787234  0.780595  0.072116  0.072037
1  Gateway-2018  RandomForest  0.542553  2.855568  0.301458  0.301379
2  Gateway-2018           SVM  0.787234  0.781698  0.072215  0.072136
3  Gateway-2018       XGBoost  0.159574  5.656009  0.577224  0.577145


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/quicktest_modules.py:1699: RuntimeWarning: invalid value encountered in less_equal
  * ((target <= quantile_forecast) - q)


In [42]:
stint_split_result['normal']

,Year,Model,SignAcc,MAE,50-Risk,90-Risk
0,Gateway-2018,CurRank,1.000000,0.036590,0.004852,0.004852
1,Gateway-2018,RandomForest,0.421488,0.975468,0.124543,0.124543
2,Gateway-2018,SVM,1.000000,0.036590,0.004852,0.004852
3,Gateway-2018,XGBoost,0.115702,2.180457,0.254939,0.254939


In [43]:
stint_split_result['pit']

,Year,Model,SignAcc,MAE,50-Risk,90-Risk
0,Gateway-2018,CurRank,0.787234,0.780595,0.072116,0.072037
1,Gateway-2018,RandomForest,0.542553,2.855568,0.301458,0.301379
2,Gateway-2018,SVM,0.787234,0.781698,0.072215,0.072136
3,Gateway-2018,XGBoost,0.159574,5.656009,0.577224,0.577145


In [44]:
dfs = []
for laptype in ['normal','pit']:
    df = stint_split_result[laptype]
    df['laptype'] = laptype
    
    dfs.append(df)
    
alldf = pd.concat(dfs)

In [45]:
alldf.to_csv(f'evaluation_result_shortterm_plen2-split-g{covergap}_v4-c{_context_len}.csv')

In [ ]:
alldf

## test new oracle

In [ ]:
oracle_split_result = {}
for laptype in ['normal','pit']:
    
    plen = 2
    
    retdata = []
    for year in ['2018']:
        
        # select the set
        pitcoveredlaps = pitdata[year][1]
        normallaps = set([x for x in range(1,201)]) - pitcoveredlaps
        
        if laptype == 'normal':
            sellaps = normallaps
            clearlaps = pitcoveredlaps
        else:
            sellaps = pitcoveredlaps
            clearlaps = normallaps
        
        
        # pitcoveredlaps start idx = 1
        startlaps = [x-plen-1 for x in sellaps]
        #sellapidx = np.array([x-1 for x in sellaps])
        clearidx = np.array([x-1 for x in clearlaps])
        print('year:%s, type:%s, lapscnt:%s'%(year, laptype, len(startlaps)))
        
        #oracle
        outfile=f'shortterm-dfout-ranknet-indy500-rank-inlap-nopitage-20182019-oracle-l10-alldata-weighted.pickle'
        _all = load_dfout_all(outfile)[0]
        ranknetdf, acc, ret, pret = _all[0],_all[1],_all[2],_all[3]

        dfout = do_rerank(ranknetdf[year]['oracle_mean'])

        allsamples, alltss = get_allsamples(year=year, model='oracle')

        allsamples, alltss = clear_samples(allsamples, alltss,clearidx)
        _, prisk_vals = prisk_direct_bysamples(allsamples, alltss)

        dfout = dfout[dfout['startlap'].isin(startlaps)]
        accret = stint.get_evalret_shortterm(dfout)[0]

        print(year, laptype,'RankNet-Oracle',accret[0], accret[1], prisk_vals[1], prisk_vals[2])
        retdata.append([year, 'RankNet-Oracle',accret[0], accret[1], prisk_vals[1], prisk_vals[2]])
        
    oracle_split_result[laptype] = pd.DataFrame(data=retdata, columns=cols)

In [ ]:
oracle_split_result

In [ ]:
dfs = []
for laptype in ['normal','pit']:
    df = oracle_split_result[laptype]
    df['laptype'] = laptype + '-weighted'
    
    dfs.append(df)
    
alldf = pd.concat(dfs)
alldf

In [ ]:
xdf = pd.read_csv(f'evaluation_result_shortterm_plen2-split-g{covergap}_v4.csv')

In [ ]:
xdf

In [ ]:
dfs.append(xdf)
df = pd.concat(dfs)
df

In [ ]:
df.to_csv(f'evaluation_result_shortterm_plen2-split-g{covergap}_v4_weighted.csv')